<a href="https://colab.research.google.com/github/ArnaudOlt/dissertation_pybamm/blob/main/main_read_lfp_balance_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pybamm
import time
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.5/67.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 22.9 MB/s eta 0:00:00


In [8]:
model = pybamm.lithium_ion.SPMe(
    {
        #"SEI": "solvent-diffusion limited",
        #"SEI porosity change": "true",
        "lithium plating": "partially reversible",
        "lithium plating porosity change": "true",  # alias for "SEI porosity change"
        #"particle mechanics": ("swelling and cracking", "swelling only"),
        #"SEI on cracks": "true",
        #"loss of active material": "stress-driven",
        #"calculate discharge energy": "true",  # for compatibility with older PyBaMM versions
    }
)

In [2]:
parameter_values = pybamm.ParameterValues("OKane2022")

In [3]:
# update parameters towards LFP cell


def LFP_ocp_ashfar2017(sto):
    """
    Open-circuit potential for LFP

    References
    ----------
    .. [1] Afshar, S., Morris, K., & Khajepour, A. (2017). Efficient electrochemical
    model for lithium-ion cells. arXiv preprint arXiv:1709.03970.

    Parameters
    ----------
    sto : :class:`pybamm.Symbol`
       Stochiometry of material (li-fraction)

    """

    c1 = -150 * sto
    c2 = -30 * (1 - sto)
    k = 3.4077 - 0.020269 * sto + 0.5 * pybamm.exp(c1) - 0.9 * pybamm.exp(c2)

    return k

def LFP_electrolyte_exchange_current_density_kashkooli2017(c_e, c_s_surf, c_s_max, T):
    """
    Exchange-current density for Butler-Volmer reactions between LFP and electrolyte

    References
    ----------
    .. [1] Kashkooli, A. G., Amirfazli, A., Farhad, S., Lee, D. U., Felicelli, S., Park,
    H. W., ... & Chen, Z. (2017). Representative volume element model of lithium-ion
    battery electrodes based on X-ray nano-tomography. Journal of Applied
    Electrochemistry, 47(3), 281-293.

    Parameters
    ----------
    c_e : :class:`pybamm.Symbol`
        Electrolyte concentration [mol.m-3]
    c_s_surf : :class:`pybamm.Symbol`
        Particle concentration [mol.m-3]
    c_s_max : :class:`pybamm.Symbol`
        Maximum particle concentration [mol.m-3]
    T : :class:`pybamm.Symbol`
        Temperature [K]

    Returns
    -------
    :class:`pybamm.Symbol`
        Exchange-current density [A.m-2]
    """

    m_ref = 6 * 10 ** (-7)  # (A/m2)(m3/mol)**1.5 - includes ref concentrations
    E_r = 39570
    arrhenius = pybamm.exp(E_r / pybamm.constants.R * (1 / 298.15 - 1 / T))

    return (
        m_ref * arrhenius * c_e**0.5 * c_s_surf**0.5 * (c_s_max - c_s_surf) ** 0.5
    )

parameter_values.update({
        "Positive electrode conductivity [S.m-1]": 0.33795074,
        "Maximum concentration in positive electrode [mol.m-3]": 22806.0,
        "Positive electrode diffusivity [m2.s-1]": 5.9e-18,
        "Positive electrode OCP [V]": LFP_ocp_ashfar2017,
        "Positive electrode porosity": 0.12728395,
        "Positive electrode active material volume fraction": 0.28485556,
        "Positive particle radius [m]": 1e-08,
        "Positive electrode Bruggeman coefficient (electrode)": 1.5,
        "Positive electrode Bruggeman coefficient (electrolyte)": 1.5,
        "Positive electrode charge transfer coefficient": 0.5,
        "Positive electrode double-layer capacity [F.m-2]": 0.2,
        "Positive electrode density [kg.m-3]": 2341.17,
        "Positive electrode specific heat capacity [J.kg-1.K-1]": 1100.0,
        "Positive electrode thermal conductivity [W.m-1.K-1]": 2.1,
        "Positive electrode OCP entropic change [V.K-1]": 0.0,
        "Positive electrode exchange-current density [A.m-2]"
        "": LFP_electrolyte_exchange_current_density_kashkooli2017,
        "Initial concentration in positive electrode [mol.m-3]": 35.3766672,
        "Lower voltage cut-off [V]": 2.0,
        "Upper voltage cut-off [V]": 4.4,
        "Open-circuit voltage at 0% SOC [V]": 2.0,
        "Open-circuit voltage at 100% SOC [V]": 4.4}, check_already_exists=False) # prada2013



In [6]:
pybamm.set_logging_level("NOTICE")
cycle_number = 500
exp = pybamm.Experiment(
    ["Hold at 3.8 V until C/20", # Changed to 3.8V
    "Rest for 4 hours",
    "Discharge at 0.1C until 2.4 V",  # initial capacity check
    "Charge at 0.3C until 3.8 V",
    "Hold at 3.8 V until C/20",]
    + [("Discharge at 1C until 2.4 V",  # ageing cycles
    "Charge at 0.3C until 3.8 V",
    "Hold at 3.8 V until C/20",)] * cycle_number
    + ["Discharge at 0.1C until 2.4 V"],  # final capacity check
    period="5 minutes"
)


In [9]:
sim = pybamm.Simulation(model, parameter_values=parameter_values, experiment=exp)

# safe solver and reduced dt_max
safe_solver = pybamm.CasadiSolver(mode="safe", dt_max=6)

# solve
start = time.monotonic()
sol = sim.solve(solver=safe_solver, calc_esoh=False)
end = time.monotonic()
print(f"Time taken: {end - start:.2f} seconds") # 76.51 seconds


2023-08-09 11:09:59.952 - [NOTICE] callbacks.on_cycle_start(172): Cycle 1/56 (1.761 ms elapsed) --------------------
NOTICE:pybamm.logger:Cycle 1/56 (1.761 ms elapsed) --------------------
2023-08-09 11:09:59.964 - [NOTICE] callbacks.on_step_start(180): Cycle 1/56, step 1/1: Hold at 4.2 V until C/100
NOTICE:pybamm.logger:Cycle 1/56, step 1/1: Hold at 4.2 V until C/100
2023-08-09 11:10:02.798 - [NOTICE] callbacks.on_cycle_start(172): Cycle 2/56 (2.848 s elapsed) --------------------
NOTICE:pybamm.logger:Cycle 2/56 (2.848 s elapsed) --------------------
2023-08-09 11:10:02.807 - [NOTICE] callbacks.on_step_start(180): Cycle 2/56, step 1/1: Rest for 4 hours
NOTICE:pybamm.logger:Cycle 2/56, step 1/1: Rest for 4 hours
The linesearch algorithm failed with too small a step.
2023-08-09 11:10:05.735 - [NOTICE] callbacks.on_cycle_start(172): Cycle 3/56 (5.784 s elapsed) --------------------
NOTICE:pybamm.logger:Cycle 3/56 (5.784 s elapsed) --------------------
2023-08-09 11:10:05.748 - [NOTICE] c